In [1]:
import numpy as np
import scipy
import plotly.graph_objects as go

## Using convolve2d

In [2]:
@np.vectorize
def next_state(state, n_neighbours):
    if state == 0 and n_neighbours == 3:
        return 1
    if state == 1 and (n_neighbours == 2 or n_neighbours == 3):
        return 1
    return 0

In [3]:
filt = np.array([
    [1, 1, 1],
    [1, 0, 1],
    [1, 1, 1]
])

In [4]:
def get_n_neighbours(board, filt):
    return scipy.signal.convolve2d(
        board, 
        filt, 
        mode='same', 
        boundary='fill', 
        fillvalue=0
    )

In [5]:
def game_of_life(board, n_iterations):
    boards = [board]
    for i in range(n_iterations-1):
        n_neighbours = get_n_neighbours(board, filt)
        board = next_state(board, n_neighbours)
        boards.append(board)
    return boards

## Using pure numpy

In [6]:
def add_zero_border(board):
    return np.pad(board, [(1, 1), (1, 1)], constant_values=0)

In [7]:
def get_n_neighbours(board):
    a = add_zero_border(board)
    return (
        a[:-2, :-2]  + a[1:-1, :-2] + a[2:, :-2]  + 
        a[:-2, 1:-1] + 0            + a[2:, 1:-1] + 
        a[:-2, 2:]   + a[1:-1, 2:]  + a[2:, 2:]
    )

In [8]:
def next_state(board, n_neighbours):
    new_life = (board == 0) & (n_neighbours == 3)
    continue_life = (board == 1) & ((n_neighbours == 2) | (n_neighbours == 3))
    return np.where(new_life|continue_life, 1, 0)

In [9]:
def game_of_life(board, n_iterations):
    boards = [board]
    for i in range(n_iterations-1):
        n_neighbours = get_n_neighbours(board)
        board = next_state(board, n_neighbours)
        boards.append(board)
    return boards

## Game

In [10]:
N = 100
p = 0.5
n_iterations = 100

board = np.random.choice([0, 1], size=[N, N], p=[1-p, p])
boards = game_of_life(board, n_iterations)

## Animate

In [11]:
fig = go.Figure(
    data=go.Heatmap(z=boards[0]),
    frames=[
        go.Frame(data=go.Heatmap(z=boards[i]))
        for i in range(n_iterations)
    ]
)

fig.update_traces(
    showscale=False,
    colorscale=['white', 'green']
)

fig.update_layout(
    height=900,
    width=900
)

fig.write_html('plots/game.html')